In [1]:
import sys; sys.path.append("C:/GitWorks/pdb-profiling")
from pdb_profiling.fetcher.webfetch import UnsyncFetch
from pdb_profiling.processers.pdbe.api import ProcessPDBe, PDBeDecoder, FUNCS
from pdb_profiling.utils import init_semaphore, split_df_by_chain, slice_series, to_interval, a_read_csv

In [2]:
from collections import Counter
import pandas as pd
import numpy as np
import orjson as json
from pathlib import Path
from unsync import unsync
from collections import defaultdict
from time import perf_counter
from typing import Iterable, Union

In [3]:
UnsyncFetch.use_existing = True
ProcessPDBe.use_existing = True
ProcessPDBe.init_logger()
semaphore = init_semaphore(30).result()

In [4]:
[j for i in FUNCS for j in i[1]]

['api/pdb/entry/cofactor/',
 'api/pdb/entry/ligand_monomers/',
 'api/pdb/entry/experiment/',
 'api/pdb/entry/mutated_AA_or_NA/',
 'api/pdb/entry/status/',
 'api/pdb/entry/summary/',
 'api/pdb/entry/drugbank/',
 'api/pdb/entry/electron_density_statistics/',
 'api/pdb/entry/related_experiment_data/',
 'api/pdb/entry/modified_AA_or_NA/',
 'api/pdb/entry/molecules/',
 'api/pdb/entry/polymer_coverage/',
 'api/pdb/entry/observed_residues_ratio/',
 'api/pdb/entry/residue_listing/',
 'api/pdb/entry/secondary_structure/',
 'api/pdb/entry/binding_sites/',
 'api/pdb/entry/assembly/',
 'api/pdb/entry/files/',
 'api/mappings/all_isoforms/',
 'api/pisa/interfacelist/',
 'api/pisa/interfacedetail/']

In [5]:
reslist_suffix = 'api/pdb/entry/residue_listing/'
eec_suffix = 'api/pdb/entry/molecules/'
obseec_suffix = 'api/pdb/entry/observed_residues_ratio/'
assembly_suffix = 'api/pdb/entry/assembly/'
interfacelist_suffix = 'api/pisa/interfacelist/'
interfacedetail_suffix = 'api/pisa/interfacedetail/'

reslist_path, eec_path, obseec_path, assembly_path, interfacelist_path, interfacedetail_path = \
    tuple(PDBeDecoder.init_folder('./demo/', (reslist_suffix, eec_suffix, obseec_suffix, assembly_suffix, interfacelist_suffix, interfacedetail_suffix)))